# Lab 03: Prompt Caching

## Overview

In this notebook, we enable **prompt caching** to reduce costs on repeated requests. Prompt caching stores the processed system prompt and tool definitions, so subsequent requests can reuse them at a 90% discount.

**What you'll learn:**
- How to configure system prompt caching
- How to enable tool definition caching
- How to verify cache hits in Langfuse
- How to calculate cost savings from caching

**Optimizations in this notebook:**
- `SystemContentBlock` with `cachePoint` (system prompt caching - provider-agnostic)
- `cache_tools="default"` on BedrockModel (tool definition caching)

## Prerequisites

- Completed Labs 01-02

## Workshop Journey

```
01 Baseline → 02 Quick Wins → [03 Caching] → 04 Routing → 05 Guardrails → 06 Gateway → 07 Evaluations
                                   ↑
                              You are here
```

## Step 1: Setup

In [1]:
from __future__ import annotations

import json
import os
import uuid
from pathlib import Path

from dotenv import load_dotenv

load_dotenv(override=True)

import boto3
from bedrock_agentcore_starter_toolkit import Runtime

region = os.environ.get("AWS_DEFAULT_REGION", "us-east-1")
control_client = boto3.client("bedrock-agentcore-control", region_name=region)
data_client = boto3.client("bedrock-agentcore", region_name=region)
agentcore_runtime = Runtime()

print(f"Region: {region}")
print(f"Langfuse Host: {os.environ.get('LANGFUSE_BASE_URL', 'Not set')}")

Region: us-east-1
Langfuse Host: https://d2rhlwziq3nnbf.cloudfront.net


## Step 2: Understanding Prompt Caching

### What is Prompt Caching?

Prompt caching stores frequently reused content so subsequent requests can skip reprocessing it. This notebook enables **two types of caching**:

| Cache Type | What's Cached | Configuration |
|------------|---------------|---------------|
| **System Prompt** | The entire system prompt (~1,100 tokens) | `SystemContentBlock` with `cachePoint` |
| **Tool Definitions** | All tool schemas and descriptions | `cache_tools="default"` on BedrockModel |

### Minimum Token Requirement

> **Important:** Claude Sonnet 4.5 requires **at least 1,024 tokens** before a cache checkpoint for caching to activate.
>
> If your system prompt is below this threshold:
> - Inference still succeeds normally (no error)
> - No cache checkpoint is created (silently skipped)
> - `cacheWriteInputTokens` and `cacheReadInputTokens` remain 0
>
> **Solution:** Expand short system prompts with few-shot examples to meet the minimum.

### System Prompt Caching

Use `SystemContentBlock` with a cache point at the end:

```python
from strands.types.content import SystemContentBlock

# Create system prompt with cache point
system_prompt = [
    SystemContentBlock(text=SYSTEM_PROMPT_TEXT),  # Must be 1,024+ tokens
    SystemContentBlock(cachePoint={"type": "default"})  # Cache checkpoint
]

agent = Agent(system_prompt=system_prompt)
```

### Tool Definition Caching

Enable tool caching via `BedrockModel`:

```python
model = BedrockModel(
    model_id="us.anthropic.claude-sonnet-4-5-20250929-v1:0",
    cache_tools="default"  # Cache all tool definitions
)
```

### Pricing

Cache pricing varies by model. See [Amazon Bedrock Pricing](https://aws.amazon.com/bedrock/pricing/) for current rates.

### Cache Metrics in Langfuse

- `cacheWriteInputTokens` - Tokens written to cache (first request after cache expires)
- `cacheReadInputTokens` - Tokens read from cache (subsequent requests within 5-min TTL)

In [2]:
# Review the caching configuration in v3 agent
agent_file = Path("agents/v3_caching.py")
print(agent_file.read_text())

"""
V3 Caching Agent - Same as v2 + prompt caching.
- All v2 optimizations (structured prompt, max_tokens, stop_sequences, low temperature)
- System prompt caching with SystemContentBlock + cachePoint (provider-agnostic)
- Tool definition caching with cache_tools="default" on BedrockModel
- Note: System prompt must be 1,024+ tokens for caching to activate
"""

import base64
import os
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from dotenv import load_dotenv
from strands import Agent
from strands.models import BedrockModel
from strands.telemetry import StrandsTelemetry
from strands.types.content import SystemContentBlock

import sys
sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
from utils.tools import get_return_policy, get_product_info, web_search, get_technical_support

load_dotenv()

# Langfuse configuration
langfuse_public_key = os.environ.get("LANGFUSE_PUBLIC_KEY")
langfuse_secret_key = os.environ.get("LANGFUSE_SECRET_KEY")
langfuse_b

## Step 3: Deploy the Caching Agent

In [3]:
agent_name = "customer_support_v3_caching"
agent_file = str(Path("agents/v3_caching.py").absolute())
requirements_file = str(Path("requirements-for-agentcore.txt").absolute())

print(f"Agent name: {agent_name}")
print(f"Agent file: {agent_file}")
print(f"Requirements: {requirements_file}")

print(f"Configuring agent: {agent_name}")
agentcore_runtime.configure(
    entrypoint=agent_file,
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file=requirements_file,
    region=region,
    agent_name=agent_name,
)

Entrypoint parsed: file=/Users/tracilim/Projects/aws-bedrock-prompt-optimization-workshop/03-developer-journey/agents/v3_caching.py, bedrock_agentcore_name=v3_caching
Memory disabled - agent will be stateless
Configuring BedrockAgentCore agent: customer_support_v3_caching


Agent name: customer_support_v3_caching
Agent file: /Users/tracilim/Projects/aws-bedrock-prompt-optimization-workshop/03-developer-journey/agents/v3_caching.py
Requirements: /Users/tracilim/Projects/aws-bedrock-prompt-optimization-workshop/03-developer-journey/requirements-for-agentcore.txt
Configuring agent: customer_support_v3_caching


💡 No container engine found (Docker/Finch/Podman not installed)

✓ Default deployment uses CodeBuild (no container engine needed), For local builds, install Docker, Finch, or 
Podman

Memory disabled
Network mode: PUBLIC
Generated Dockerfile: Dockerfile
Generated .dockerignore: /Users/tracilim/Projects/aws-bedrock-prompt-optimization-workshop/03-developer-journey/.dockerignore
Keeping 'customer_support_v3_caching' as default agent
Bedrock AgentCore configured: /Users/tracilim/Projects/aws-bedrock-prompt-optimization-workshop/03-developer-journey/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/Users/tracilim/Projects/aws-bedrock-prompt-optimization-workshop/03-developer-journey/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/Users/tracilim/Projects/aws-bedrock-prompt-optimization-workshop/03-developer-journey/Dockerfile'), dockerignore_path=PosixPath('/Users/tracilim/Projects/aws-bedrock-prompt-optimization-workshop/03-developer-journey/.dockerignore'), runtime='None', runtime_type=None, region='us-east-1', account_id='739907928487', execution_role=None, ecr_repository=None, auto_create_ecr=True, s3_path=None, auto_create_s3=False, memory_id=None, network_mode='PUBLIC', network_subnets=None, network_security_groups=None, network_vpc_id=None)

In [4]:
# Modify Dockerfile for Langfuse
dockerfile_path = Path("Dockerfile")
if dockerfile_path.exists():
    content = dockerfile_path.read_text()
    # Replace opentelemetry-instrument wrapper with direct python call
    # Keep the correct module path using regex
    if "opentelemetry-instrument" in content:
        import re
        content = re.sub(
            r'CMD \["opentelemetry-instrument", "python", "-m", "([^"]+)"\]',
            r'CMD ["python", "-m", "\1"]',
            content
        )
        dockerfile_path.write_text(content)
        print("Dockerfile modified for Langfuse")
    else:
        print("Dockerfile already configured or using different format")
else:
    print("Dockerfile not found - will be created during deployment")

Dockerfile modified for Langfuse


In [5]:
env_vars = {
    "LANGFUSE_BASE_URL": os.environ.get("LANGFUSE_BASE_URL"),
    "LANGFUSE_PUBLIC_KEY": os.environ.get("LANGFUSE_PUBLIC_KEY"),
    "LANGFUSE_SECRET_KEY": os.environ.get("LANGFUSE_SECRET_KEY"),
    "PYTHONUNBUFFERED": "1",
}

print("Deploying to AgentCore Runtime...")
launch_result = agentcore_runtime.launch(env_vars=env_vars, auto_update_on_conflict=True)
agent_arn = launch_result.agent_arn
print(f"Agent deployed: {agent_arn}")

🚀 Launching Bedrock AgentCore (cloud mode - RECOMMENDED)...
   • Deploy Python code directly to runtime
   • No Docker required (DEFAULT behavior)
   • Production-ready deployment

💡 Deployment options:
   • runtime.launch()                → Cloud (current)
   • runtime.launch(local=True)      → Local development
Memory disabled - skipping memory creation
Starting CodeBuild ARM64 deployment for agent 'customer_support_v3_caching' to account 739907928487 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: customer_support_v3_caching


Deploying to AgentCore Runtime...


ECR repository available: 739907928487.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-customer_support_v3_caching
Getting or creating execution role for agent: customer_support_v3_caching
Using AWS region: us-east-1, account ID: 739907928487
Role name: AmazonBedrockAgentCoreSDKRuntime-us-east-1-9236411bd3


✅ Reusing existing ECR repository: 739907928487.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-customer_support_v3_caching


✅ Reusing existing execution role: arn:aws:iam::739907928487:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-9236411bd3
Execution role available: arn:aws:iam::739907928487:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-9236411bd3
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: customer_support_v3_caching
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-9236411bd3
Reusing existing CodeBuild execution role: arn:aws:iam::739907928487:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-9236411bd3
Using dockerignore.template with 46 patterns for zip filtering
Uploaded source to S3: customer_support_v3_caching/source.zip
Updated CodeBuild project: bedrock-agentcore-customer_support_v3_caching-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.3s
🔄 PROVISIONING started (total: 2s)
✅ PROVISIONING completed in 9.0s
🔄 DOWNLOAD_SO

Agent deployed: arn:aws:bedrock-agentcore:us-east-1:739907928487:runtime/customer_support_v3_caching-spXyyp5y1x


In [6]:
agent_arn = 'arn:aws:bedrock-agentcore:us-east-1:739907928487:runtime/customer_support_v3_caching-spXyyp5y1x'

## Step 4: Test Caching Behavior

Run the standard test prompts and observe cache metrics in Langfuse.

**What to look for:**
- `Cache Read Tokens` > 0 indicates the system prompt and tool definitions are being read from cache
- `Cache Write Tokens` > 0 indicates tokens were written to cache (only happens when cache is cold/expired)

In [7]:
def invoke_agent(prompt):
    """Invoke the agent via AgentCore API."""
    response = data_client.invoke_agent_runtime(
        agentRuntimeArn=agent_arn,
        runtimeSessionId=str(uuid.uuid4()),
        payload=json.dumps({"prompt": prompt}).encode(),
    )
    return json.loads(response["response"].read().decode("utf-8"))

In [8]:
# Import Langfuse metrics helper
from utils.langfuse_metrics import (
    clear_metrics,
    collect_metric,
    get_latest_trace_metrics,
    print_metrics,
    print_metrics_table,
)

# Clear any previously collected metrics
clear_metrics()

# Standard test prompts - same across all notebooks for consistent comparison
TEST_PROMPTS = [
    # Single tool: get_return_policy
    ("Return Policy", "What is your return policy for laptops?"),

    # Single tool: get_product_info
    ("Product Info", "Tell me about your smartphone options"),

    # Single tool: get_technical_support (Bedrock KB)
    ("Technical Support", "My laptop won't turn on, can you help me troubleshoot?"),

    # Multi-tool: get_product_info + get_return_policy
    ("Multi-part Question", "I want to buy a laptop. What are the specs and what's the return policy?"),

    # No tool: General greeting
    ("General Question", "Hello! What can you help me with today?"),
]

# Run all tests and collect metrics
# Cache behavior: First request after deployment writes to cache, subsequent requests read from cache
# If cache is already warm (within 5-min TTL), all requests will show cache reads
for i, (test_name, prompt) in enumerate(TEST_PROMPTS):
    print("=" * 60)
    print(f"Test {i+1}: {test_name}")
    print("=" * 60)

    response = invoke_agent(prompt)
    print(response)

    # Fetch and collect metrics
    metrics = get_latest_trace_metrics(
        agent_name="customer-support-v3-caching",
        wait_seconds=5,
        max_retries=5,
        timeout_seconds=120,
    )
    print_metrics(metrics, test_name)
    collect_metric(metrics, test_name)

Test 1: Return Policy
- **answer:** Our laptop return policy offers flexibility with the following details:

  **Return Window:** 30 days from purchase date
  
  **Requirements:**
  - Original packaging with all accessories included
  - No physical damage to the device
  - Receipt or order confirmation
  
  **Return Process:**
  - Submit through our online RMA portal, or
  - Bring to any TechMart store location
  
  **Refund Timeline:** 7-10 business days after we inspect the returned item
  
  **Shipping Costs:**
  - FREE return shipping for defective items
  - Customer pays shipping for change-of-mind returns
  
  **Restocking Fees:**
  - No fee for defective products
  - 15% restocking fee for change-of-mind returns
  
  **Warranty:** All laptops include a 1-year manufacturer warranty, with extended warranty options available at purchase.

  Would you like help starting a return or do you have questions about a specific laptop purchase?

- **category:** policy
- **confidence:** high

In [9]:
# Print summary table
print_metrics_table()


                                  METRICS SUMMARY
               Test Latency    Cost Input Output Cache Read Tokens Cache Write Tokens
      Return Policy   8.15s $0.0077   863    306             1,743              1,743
       Product Info   7.34s $0.0091   910    353             3,486                  0
  Technical Support  12.07s $0.0110   916    480             3,486                  0
Multi-part Question   9.70s $0.0125 1,207    519             3,486                  0
   General Question   3.25s $0.0036   332    137             1,743                  0
---------------------------------------------------------------------------------------------------------
  TOTALS: Latency(avg): 8.10s | Cost: $0.0438 | Input: 4,228 | Output: 1,795
          Cache Read Tokens: 13,944 | Cache Write Tokens: 1,743



,Test,Latency,Cost,Input,Output,Cache Read Tokens,Cache Write Tokens
0,Return Policy,8.15s,$0.0077,863,306,"1,743","1,743"
1,Product Info,7.34s,$0.0091,910,353,"3,486",0
2,Technical Support,12.07s,$0.0110,916,480,"3,486",0
3,Multi-part Question,9.70s,$0.0125,"1,207",519,"3,486",0
4,General Question,3.25s,$0.0036,332,137,"1,743",0


## Step 5: Compare with v2 (Quick Wins)

Enter your metrics from Lab 02 (v2 quick wins) to compare cost, latency, and token usage.

In [ ]:
from utils.langfuse_metrics import calculate_totals_from_collected, print_comparison

# ============================================================
# INPUT YOUR V2 METRICS FROM LAB 02 HERE
# (Copy the totals from your v2 metrics table)
# ============================================================
v2_total_cost = 0.0858       # e.g., 0.0858
v2_avg_latency = 7.30        # e.g., 7.30 (seconds)
v2_total_input_tokens = 19920    # e.g., 19920
v2_total_output_tokens = 1737   # e.g., 1737

# Print comparison (current metrics auto-calculated from collected)
print_comparison(
    prev_name="v2 (Quick Wins)",
    curr_name="v3 (Caching)",
    prev_cost=v2_total_cost,
    prev_latency=v2_avg_latency,
    prev_input_tokens=v2_total_input_tokens,
    prev_output_tokens=v2_total_output_tokens,
)

# Show cache-specific metrics (unique to v3)
totals = calculate_totals_from_collected()
print("\nCache Metrics (v3 only):")
print(f"  Cache Read Tokens:  {totals['total_cache_read_tokens']:,}")
print(f"  Cache Write Tokens: {totals['total_cache_write_tokens']:,}")

## Summary

In this notebook, we enabled **two types of caching** on top of the v2 optimizations:

| Cache Type | Configuration | Tokens Cached |
|------------|---------------|---------------|
| **System Prompt** | `SystemContentBlock` + `cachePoint` | ~1,100 tokens |
| **Tool Definitions** | `cache_tools="default"` | ~1,100 tokens |

```python
from strands.types.content import SystemContentBlock

# 1. System prompt caching
system_prompt = [
    SystemContentBlock(text=SYSTEM_PROMPT_TEXT),  # Must be 1,024+ tokens
    SystemContentBlock(cachePoint={"type": "default"})
]

# 2. Tool definition caching
model = BedrockModel(
    cache_tools="default",
    ...
)

agent = Agent(model=model, system_prompt=system_prompt)
```

**Key Observations:**
- First request after cache expires writes to cache (premium pricing)
- Subsequent requests read from cache (discounted pricing)
- The 5-minute TTL refreshes on each cache hit
- System prompt must be **at least 1,024 tokens** for caching

**When to use caching:**
- Agents with consistent traffic (requests within 5-minute windows)
- Static system prompts and tool definitions
- High-volume production workloads

**Pricing:** See [Amazon Bedrock Pricing](https://aws.amazon.com/bedrock/pricing/) for current cache read/write rates.

---

**Next:** In Lab 04, we'll explore **model routing** to use cheaper models for simple queries.

**Next notebook:** [04-llm-routing.ipynb](./04-llm-routing.ipynb)

---

## Cleanup

To delete the agent deployed in this notebook, uncomment and run the following code.

In [11]:
# # Delete the agent
# control_client.delete_agent_runtime(agentRuntimeId=agent_arn.split("/")[-1])
# print(f"Agent deleted: {agent_arn}")